In [113]:
#pip install deepface

In [25]:
import os

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns

import PIL 
from PIL import Image

import cv2
import time

from deepface import DeepFace

import time

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
!nvidia-smi
torch.cuda.is_available()

Wed Mar  5 04:01:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 561.19                 Driver Version: 561.19         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   59C    P0             13W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

True

In [4]:
#настроим random_seed всего
seed_num = 13
torch.manual_seed(seed_num)
torch.cuda.manual_seed(seed_num)
torch.cuda.manual_seed_all(seed_num)
torch.use_deterministic_algorithms(True) # do not work in collab
np.random.seed(seed_num)

import torch.backends.cudnn
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True # do not work in collab

In [5]:
project_path = r'F:\Codenrock\Kryptonite'
local_path = os.getcwd()
project_path

'F:\\Codenrock\\Kryptonite'

### Try model

DeepFace https://github.com/serengil/deepface/tree/master

##### При запуске первый раз модель скачивает веса. Потом работает уже быстро

##### Первый режим работы

In [11]:
image_path = os.path.join(project_path, 'data', 'train', 'images', '000000')

result = DeepFace.verify(
  img1_path = os.path.join(image_path, "0.jpg"),
  img2_path = os.path.join(image_path, "1.jpg"),
)

In [17]:
%%time 

result = DeepFace.verify(
  img1_path = os.path.join(image_path, "0.jpg"),
  img2_path = os.path.join(image_path, "1.jpg"),
)

CPU times: total: 1.84 s
Wall time: 687 ms


In [17]:
result

{'verified': True,
 'distance': 0.38093279256577395,
 'threshold': 0.68,
 'model': 'VGG-Face',
 'detector_backend': 'opencv',
 'similarity_metric': 'cosine',
 'facial_areas': {'img1': {'x': 15,
   'y': 14,
   'w': 202,
   'h': 202,
   'left_eye': None,
   'right_eye': None},
  'img2': {'x': 15,
   'y': 26,
   'w': 191,
   'h': 191,
   'left_eye': None,
   'right_eye': None}},
 'time': 2.58}

In [23]:
result['distance']

0.38093279256577395

In [40]:
# измерим время инференса

start_time = time.time()

result = DeepFace.verify(
  img1_path = os.path.join(image_path, "0.jpg"),
  img2_path = os.path.join(image_path, "1.jpg"),
)
# конечное время
end_time = time.time()
print('Inference time: ', end_time - start_time)

Inference time:  0.6655762195587158


In [17]:
# измерим время инференса, если перенесем фото с внешнего диска на С
local_image_path = r'C:\Users\serge\DS\Codenrock\Kryptonite\000000'
start_time = time.time()

result = DeepFace.verify(
  img1_path = os.path.join(local_image_path, "0.jpg"),
  img2_path = os.path.join(local_image_path, "1.jpg"),
)
# конечное время
end_time = time.time()
print('Inference time: ', end_time - start_time)

Inference time:  0.6288270950317383


Время инференса меняется от запуска к запуску на одной и той же паре фото. Составляет от 0.6 до 1.2. Перенос с внешнего диска особо скорости не добавил

##### Второй режим работы

In [108]:
embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "0.jpg")
)

for embedding_obj in embedding_objs:
  embedding_0 = np.array(embedding_obj["embedding"])
embedding_0 

array([0., 0., 0., ..., 0., 0., 0.])

In [110]:
  img_path = os.path.join(local_image_path, "1.jpg")
)

for embedding_obj in embedding_objs:
  embedding_1 = np.array(embedding_obj["embedding"])
embedding_1

array([0., 0., 0., ..., 0., 0., 0.])

In [114]:
sim_scores = cosine_similarity(embedding_0.reshape(1, -1), embedding_1.reshape(1, -1))
sim_scores

array([[0.61906721]])

In [116]:
1-0.38093279256577395

0.619067207434226

##### Результат обоими способами совпадает:)

In [178]:
# check time
start_time = time.time()

embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "0.jpg")
)
for embedding_obj in embedding_objs:
  embedding_0 = np.array(embedding_obj["embedding"])

embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "1.jpg") # real
)

for embedding_obj in embedding_objs:
  embedding_1 = np.array(embedding_obj["embedding"])

sim_scores = cosine_similarity(embedding_0.reshape(1, -1), embedding_1.reshape(1, -1))

# конечное время
end_time = time.time()
print('Inference time: ', end_time - start_time)
print(sim_scores)

Inference time:  0.6533646583557129
[[0.61906721]]


Время тоже меняется, нужны массовые замеры, чтобы посчитать среднее

### Посмотрим сравнение real face и deepfake глазами

In [180]:
# check time CPU
start_time = time.time()

embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "0.jpg")
)
for embedding_obj in embedding_objs:
  embedding_0 = np.array(embedding_obj["embedding"])

embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "2.jpg") # fake
)

for embedding_obj in embedding_objs:
  embedding_1 = np.array(embedding_obj["embedding"])

sim_scores = cosine_similarity(embedding_0.reshape(1, -1), embedding_1.reshape(1, -1))

end_time = time.time()
print('Inference time: ', end_time - start_time)
print(sim_scores)

Inference time:  0.6491076946258545
[[0.6079998]]


In [182]:
# check time CPU
start_time = time.time()

embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "0.jpg")
)
for embedding_obj in embedding_objs:
  embedding_0 = np.array(embedding_obj["embedding"])

embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "3.jpg") # fake
)

for embedding_obj in embedding_objs:
  embedding_1 = np.array(embedding_obj["embedding"])

sim_scores = cosine_similarity(embedding_0.reshape(1, -1), embedding_1.reshape(1, -1))

end_time = time.time()
print('Inference time: ', end_time - start_time)
print(sim_scores)

Inference time:  0.646836519241333
[[0.43059926]]


In [184]:
# check time CPU
start_time = time.time()

embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "0.jpg")
)
for embedding_obj in embedding_objs:
  embedding_0 = np.array(embedding_obj["embedding"])

embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "4.jpg") # real
)

for embedding_obj in embedding_objs:
  embedding_1 = np.array(embedding_obj["embedding"])

sim_scores = cosine_similarity(embedding_0.reshape(1, -1), embedding_1.reshape(1, -1))

end_time = time.time()
print('Inference time: ', end_time - start_time)
print(sim_scores)

Inference time:  0.7515063285827637
[[0.55766013]]


In [186]:
# check time CPU
start_time = time.time()

embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "0.jpg")
)
for embedding_obj in embedding_objs:
  embedding_0 = np.array(embedding_obj["embedding"])

embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "5.jpg") # real
)

for embedding_obj in embedding_objs:
  embedding_1 = np.array(embedding_obj["embedding"])

sim_scores = cosine_similarity(embedding_0.reshape(1, -1), embedding_1.reshape(1, -1))

end_time = time.time()
print('Inference time: ', end_time - start_time)
print(sim_scores)

Inference time:  0.6569499969482422
[[0.62218966]]


In [188]:
# check time CPU
start_time = time.time()

embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "0.jpg")
)
for embedding_obj in embedding_objs:
  embedding_0 = np.array(embedding_obj["embedding"])

embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "6.jpg") # fake
)

for embedding_obj in embedding_objs:
  embedding_1 = np.array(embedding_obj["embedding"])

sim_scores = cosine_similarity(embedding_0.reshape(1, -1), embedding_1.reshape(1, -1))

end_time = time.time()
print('Inference time: ', end_time - start_time)
print(sim_scores)

Inference time:  0.661942720413208
[[0.44014376]]


In [190]:
# check time CPU
start_time = time.time()

embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "0.jpg")
)
for embedding_obj in embedding_objs:
  embedding_0 = np.array(embedding_obj["embedding"])

embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "7.jpg") # fake
)

for embedding_obj in embedding_objs:
  embedding_1 = np.array(embedding_obj["embedding"])

sim_scores = cosine_similarity(embedding_0.reshape(1, -1), embedding_1.reshape(1, -1))

end_time = time.time()
print('Inference time: ', end_time - start_time)
print(sim_scores)

Inference time:  0.6849393844604492
[[0.5188896]]


In [192]:
# check time CPU
start_time = time.time()

embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "0.jpg")
)
for embedding_obj in embedding_objs:
  embedding_0 = np.array(embedding_obj["embedding"])

embedding_objs = DeepFace.represent(
  img_path = os.path.join(local_image_path, "8.jpg") # fake
)

for embedding_obj in embedding_objs:
  embedding_1 = np.array(embedding_obj["embedding"])

sim_scores = cosine_similarity(embedding_0.reshape(1, -1), embedding_1.reshape(1, -1))

end_time = time.time()
print('Inference time: ', end_time - start_time)
print(sim_scores)

Inference time:  0.6550729274749756
[[0.53975732]]


In [194]:
len(embedding_1)

4096

Real vs Fake в этой папке оценивается ниже, чем Real vs Real. Чтобы полноценно это доказать, нужно сравнение по трейновому датасету.
Но скорость инференса крайне низкая. Побробуем подобрать бэкбоун побыстрее.

### Backbone Comparison

In [19]:
imgs_comparison_list = os.listdir(local_image_path)
len(imgs_comparison_list)

9

In [21]:
def compare_imgs(model, path):
    res = []
    embedding_objs = DeepFace.represent(img_path = os.path.join(path, "0.jpg"), enforce_detection = False, model_name = model)
    for embedding_obj in embedding_objs:
      embedding_0 = np.array(embedding_obj["embedding"])    
    for img in range(1, len(imgs_comparison_list)):
        start_time = time.time()   
        name = str(img) + '.jpg'
        embedding_objs = DeepFace.represent(img_path = os.path.join(path, name), enforce_detection = False, model_name = model)
        for embedding_obj in embedding_objs:
          embedding_1 = np.array(embedding_obj["embedding"])
        sim_scores = cosine_similarity(embedding_0.reshape(1, -1), embedding_1.reshape(1, -1))
        end_time = time.time()
        res.append([name, sim_scores[0][0], end_time - start_time])
    return pd.DataFrame(res, columns = ['img_2', 'cos_sim', 'time'])

In [37]:
vgg_res = compare_imgs("VGG-Face", local_image_path)
vgg_res

,img_2,cos_sim,time
0,1.jpg,0.619067,0.283330
1,2.jpg,0.608000,0.321746
2,3.jpg,0.430599,0.302622
3,4.jpg,0.557660,0.325660
4,5.jpg,0.622190,0.288249
5,6.jpg,0.440144,0.344581
6,7.jpg,0.518890,0.283063
7,8.jpg,0.539757,0.300061


In [38]:
face_net_res = compare_imgs("Facenet", local_image_path)
face_net_res

,img_2,cos_sim,time
0,1.jpg,0.795937,0.233232
1,2.jpg,0.666713,0.249881
2,3.jpg,0.698489,0.250910
3,4.jpg,0.807505,0.247659
4,5.jpg,0.842795,0.250927
5,6.jpg,0.704719,0.233027
6,7.jpg,0.742352,0.249872
7,8.jpg,0.762128,0.272656


In [39]:
face_net_512_res = compare_imgs("Facenet512", local_image_path)
face_net_512_res

,img_2,cos_sim,time
0,1.jpg,0.810955,0.249847
1,2.jpg,0.695080,0.255564
2,3.jpg,0.679240,0.236072
3,4.jpg,0.809689,0.241143
4,5.jpg,0.870042,0.249858
5,6.jpg,0.825052,0.241378
6,7.jpg,0.809848,0.244299
7,8.jpg,0.825844,0.265758


In [41]:
open_face_res = compare_imgs("OpenFace", local_image_path)
open_face_res

,img_2,cos_sim,time
0,1.jpg,0.621508,0.123224
1,2.jpg,0.625966,0.108740
2,3.jpg,0.596473,0.099964
3,4.jpg,0.786489,0.116575
4,5.jpg,0.631177,0.119626
5,6.jpg,0.352457,0.113519
6,7.jpg,0.674120,0.116653
7,8.jpg,0.678561,0.133218


In [42]:
deep_id_res = compare_imgs("DeepID", local_image_path)
deep_id_res

,img_2,cos_sim,time
0,1.jpg,0.977524,0.038991
1,2.jpg,0.919044,0.042475
2,3.jpg,0.919890,0.029066
3,4.jpg,0.973824,0.042246
4,5.jpg,0.980442,0.043900
5,6.jpg,0.790755,0.040056
6,7.jpg,0.889377,0.039062
7,8.jpg,0.975122,0.057970


In [45]:
arc_face_res = compare_imgs("ArcFace", local_image_path)
arc_face_res

,img_2,cos_sim,time
0,1.jpg,0.736856,0.196290
1,2.jpg,0.583269,0.197044
2,3.jpg,0.531124,0.203160
3,4.jpg,0.803240,0.196977
4,5.jpg,0.731705,0.196247
5,6.jpg,0.651886,0.199785
6,7.jpg,0.633849,0.200263
7,8.jpg,0.673043,0.222380


In [31]:
SFace_res = compare_imgs("SFace", local_image_path)
SFace_res

25-03-05 04:05:23 - face_recognition_sface_2021dec.onnx weights will be downloaded...


Downloading...
From: https://github.com/opencv/opencv_zoo/raw/main/models/face_recognition_sface/face_recognition_sface_2021dec.onnx
To: C:\Users\serge\.deepface\weights\face_recognition_sface_2021dec.onnx
100%|██████████| 38.7M/38.7M [00:03<00:00, 9.76MB/s]


,img_2,cos_sim,time
0,1.jpg,0.500815,0.049927
1,2.jpg,0.577125,0.049970
2,3.jpg,0.449707,0.049855
3,4.jpg,0.710185,0.040926
4,5.jpg,0.633875,0.054073
5,6.jpg,0.497414,0.045856
6,7.jpg,0.571472,0.042444
7,8.jpg,0.498756,0.066701


In [33]:
ghost_res = compare_imgs("GhostFaceNet", local_image_path)
ghost_res

25-03-05 04:05:52 - Pre-trained weights is downloaded from https://github.com/HamadYA/GhostFaceNets/releases/download/v1.2/GhostFaceNet_W1.3_S1_ArcFace.h5 to C:\Users\serge\.deepface/weights/ghostfacenet_v1.h5


Downloading...
From: https://github.com/HamadYA/GhostFaceNets/releases/download/v1.2/GhostFaceNet_W1.3_S1_ArcFace.h5
To: C:\Users\serge\.deepface\weights\ghostfacenet_v1.h5
100%|██████████| 17.3M/17.3M [00:02<00:00, 7.68MB/s]


25-03-05 04:05:55 - Pre-trained weights is just downloaded to C:\Users\serge\.deepface/weights/ghostfacenet_v1.h5


,img_2,cos_sim,time
0,1.jpg,0.330522,0.200233
1,2.jpg,0.552727,0.216774
2,3.jpg,0.565779,0.215912
3,4.jpg,0.550898,0.234051
4,5.jpg,0.616968,0.248986
5,6.jpg,0.492213,0.216546
6,7.jpg,0.486151,0.199950
7,8.jpg,0.527255,0.216736


In [59]:
total_res = pd.DataFrame(columns = ['img', 'status', 'vgg_res', 'face_net_res', 'face_net_512_res', 'open_face_res', 'deep_id_res', 
                                    'arc_face_res', 'SFace_res', 'ghost_res'])
total_res['img'] = vgg_res['img_2']
total_res['status'] = [0, 1, 1, 0, 0, 1, 1, 1]
total_res['vgg_res'] = vgg_res['cos_sim']
total_res['face_net_res'] = face_net_res['cos_sim']
total_res['face_net_512_res'] = face_net_512_res['cos_sim']
total_res['open_face_res'] = open_face_res['cos_sim']
total_res['deep_id_res'] = deep_id_res['cos_sim']
total_res['arc_face_res'] = arc_face_res['cos_sim']
total_res['SFace_res'] = SFace_res['cos_sim']
total_res['ghost_res'] = ghost_res['cos_sim']

total_res

,img,status,vgg_res,face_net_res,face_net_512_res,open_face_res,deep_id_res,arc_face_res,SFace_res,ghost_res
0,1.jpg,0,0.619067,0.795937,0.810955,0.621508,0.977524,0.736856,0.500815,0.330522
1,2.jpg,1,0.608000,0.666713,0.695080,0.625966,0.919044,0.583269,0.577125,0.552727
2,3.jpg,1,0.430599,0.698489,0.679240,0.596473,0.919890,0.531124,0.449707,0.565779
3,4.jpg,0,0.557660,0.807505,0.809689,0.786489,0.973824,0.803240,0.710185,0.550898
4,5.jpg,0,0.622190,0.842795,0.870042,0.631177,0.980442,0.731705,0.633875,0.616968
5,6.jpg,1,0.440144,0.704719,0.825052,0.352457,0.790755,0.651886,0.497414,0.492213
6,7.jpg,1,0.518890,0.742352,0.809848,0.674120,0.889377,0.633849,0.571472,0.486151
7,8.jpg,1,0.539757,0.762128,0.825844,0.678561,0.975122,0.673043,0.498756,0.527255


In [61]:
total_res_time = pd.DataFrame(columns = ['img', 'vgg_res', 'face_net_res', 'face_net_512_res', 'open_face_res', 'deep_id_res', 
                                    'arc_face_res', 'SFace_res', 'ghost_res'])
total_res_time['img'] = vgg_res['img_2']
total_res_time['vgg_res'] = vgg_res['time']
total_res_time['face_net_res'] = face_net_res['time']
total_res_time['face_net_512_res'] = face_net_512_res['time']
total_res_time['open_face_res'] = open_face_res['time']
total_res_time['deep_id_res'] = deep_id_res['time']
total_res_time['arc_face_res'] = arc_face_res['time']
total_res_time['SFace_res'] = SFace_res['time']
total_res_time['ghost_res'] = ghost_res['time']

total_res_time

,img,vgg_res,face_net_res,face_net_512_res,open_face_res,deep_id_res,arc_face_res,SFace_res,ghost_res
0,1.jpg,0.283330,0.233232,0.249847,0.123224,0.038991,0.196290,0.049927,0.200233
1,2.jpg,0.321746,0.249881,0.255564,0.108740,0.042475,0.197044,0.049970,0.216774
2,3.jpg,0.302622,0.250910,0.236072,0.099964,0.029066,0.203160,0.049855,0.215912
3,4.jpg,0.325660,0.247659,0.241143,0.116575,0.042246,0.196977,0.040926,0.234051
4,5.jpg,0.288249,0.250927,0.249858,0.119626,0.043900,0.196247,0.054073,0.248986
5,6.jpg,0.344581,0.233027,0.241378,0.113519,0.040056,0.199785,0.045856,0.216546
6,7.jpg,0.283063,0.249872,0.244299,0.116653,0.039062,0.200263,0.042444,0.199950
7,8.jpg,0.300061,0.272656,0.265758,0.133218,0.057970,0.222380,0.066701,0.216736


##### В целом, все модели оценивали Real vs Fake ниже, чем Real vs Real. Возможно, можно это использовать - в модели есть параметр threshold.
Наивысшие cos_sim показал deep_id_res, он же был самым быстрым. Попробуем сделать выгрузку на нем. 

### Inference on pretrained DeepFace: deep_id_res

In [68]:
ssd_path = r'F:\Codenrock\Kryptonite\data\test_public\images'
ssd_list_dir = os.listdir(ssd_path)
ssd_list_dir[0]

'00000000'

In [79]:
cos_sims = []
start_time = time.time()
beg_time = start_time
for folder in ssd_list_dir:    
    image_path = os.path.join(ssd_path, folder)
    embedding_objs = DeepFace.represent(img_path = os.path.join(image_path, "0.jpg"), enforce_detection = False, model_name = 'DeepID')
    for embedding_obj in embedding_objs:
      embedding_0 = np.array(embedding_obj["embedding"])    
    embedding_objs = DeepFace.represent(img_path = os.path.join(image_path, "1.jpg"), enforce_detection = False, model_name = 'DeepID')    
    for embedding_obj in embedding_objs:
      embedding_1 = np.array(embedding_obj["embedding"])    
    sim_scores = cosine_similarity(embedding_0.reshape(1, -1), embedding_1.reshape(1, -1))
    cos_sims.append([folder, sim_scores])
    if int(folder) % 1000 == 0:
        print(f'Current time: {folder} ', time.time() - beg_time)  
        beg_time = time.time()

end_time = time.time()
print('Inference time: ', end_time - start_time)

Current time: 00000000  0.1233069896697998
Current time: 00001000  99.38783764839172
Current time: 00002000  99.2624900341034
Current time: 00003000  100.33959436416626
Current time: 00004000  99.13906502723694
Current time: 00005000  103.79435133934021
Current time: 00006000  103.86499977111816
Current time: 00007000  102.40512251853943
Current time: 00008000  100.63918590545654
Current time: 00009000  102.40499877929688
Current time: 00010000  103.05476927757263
Current time: 00011000  102.80469465255737
Current time: 00012000  107.45202708244324
Current time: 00013000  103.55402636528015
Current time: 00014000  105.6198673248291
Current time: 00015000  105.00326585769653
Current time: 00016000  103.93748831748962
Current time: 00017000  105.56974530220032
Current time: 00018000  102.5714328289032
Current time: 00019000  106.6368842124939
Current time: 00020000  101.1949052810669
Current time: 00021000  107.94578671455383
Current time: 00022000  105.37068367004395
Current time: 00023

In [83]:
len(cos_sims)

161191

In [87]:
preds = pd.DataFrame(cos_sims, columns = ['pair_id', 'similarity'])
preds

,pair_id,similarity
0,00000000,[[1.0]]
1,00000001,[[0.8499120443980509]]
2,00000002,[[0.7018126678136867]]
3,00000003,[[0.8081231677031278]]
4,00000004,[[0.869009833958676]]
...,...,...
161186,00161186,[[0.9814701313950365]]
161187,00161187,[[0.9503068789174927]]
161188,00161188,[[0.9558576937296217]]
161189,00161189,[[0.8838035792604774]]


In [101]:
sim_np = preds['similarity']
sim_single = []
for sim in sim_np:
    sim_single.append(sim[0][0])

In [105]:
preds['similarity'] = sim_single
preds

,pair_id,similarity
0,00000000,1.000000
1,00000001,0.849912
2,00000002,0.701813
3,00000003,0.808123
4,00000004,0.869010
...,...,...
161186,00161186,0.981470
161187,00161187,0.950307
161188,00161188,0.955858
161189,00161189,0.883804


In [109]:
preds.to_csv('submission_v0.csv', index = False)

In [111]:
preds.loc[preds['similarity']<0.5]

,pair_id,similarity
21589,00021589,0.491716
58591,00058591,0.417575
58607,00058607,0.430657
91027,00091027,0.362581
91050,00091050,0.439927
102837,00102837,0.426483
102860,00102860,0.488667
135237,00135237,0.477057
146018,00146018,0.470802
147637,00147637,0.411685


##### Фотографий разных людей, по мнению модели, было 13:)